In [110]:
# import module

import pickle
import numpy as np
import sys
from tqdm import tqdm
import math

from sklearn.cluster import DBSCAN
import hashlib

from GIPS.core.utils import minHash
from GIPS.core.JIG import JIG

In [111]:
with open('./datasets/pkl/pe_miner_key.pkl', 'rb') as f:
    miner_key = pickle.load(f)

In [112]:
with open('./datasets/pkl/pe_miner_data.pkl', 'rb') as f:
    miner_data = pickle.load(f)

In [118]:
# data 예시

print(miner_data[100]['label'])
for i in range(len(miner_key)):
    k = miner_key[i]
    v = miner_data[100]['analy'][k]
    print(f'{k}:\t {v}')

W32/Agent.OLAM-8712
SLENH.DLL:	 0
SHLWAPI.DLL:	 0
AWFXAB32.DLL:	 0
DSKMAINT.DLL:	 0
BHNETB.DLL:	 0
GROUP.DLL:	 0
AWRESX32.DLL:	 0
KERNEL32.DLL:	 1
RPCTLC6.DLL:	 0
RPCRT4.DLL:	 0
RPCTLS3.DLL:	 0
RPCTLS5.DLL:	 0
ADVAP132.DLL:	 0
MSNET32.DLL:	 0
WININET.DLL:	 0
CCAPI.DLL:	 0
WINREG.DLL:	 0
AWPWD32.DLL:	 0
NWNET32.DLL:	 0
USER32.DLL:	 1
CCEI.DLL:	 0
NETDI.DLL:	 0
MAPI32.DLL:	 0
CCPSH.DLL:	 0
MSPST32.DLL:	 0
DCIMAN.DLL:	 0
LZ32.DLL:	 0
NAL.DLL:	 0
CRTDLL.DLL:	 0
DCIMAN32.DLL:	 0
RASAPI32.DLL:	 0
COMCTL32.DLL:	 0
BHSUPP.DLL:	 0
OLEDLG.DLL:	 0
AWUTIL32.DLL:	 0
NETBIOS.DLL:	 0
RASPI.DLL:	 0
MSFS32.DLL:	 0
AWFAXP32.DLL:	 0
MSVIEWUT.DLL:	 0
RPCTLS6.DLL:	 0
VERSION.DLL:	 0
MSNDUI.DLL:	 0
NWAB32.DLL:	 0
SAPNSP.DLL:	 0
CCTN20.DLL:	 0
UMDM32.DLL:	 0
CMC.DLL:	 0
NDIS30.DLL:	 0
NETAPI32.DLL:	 0
GDI32.DLL:	 0
WINMM.DLL:	 0
NWNP32.DLL:	 0
WINSOCK.DLL:	 0
RASAPI16.DLL:	 0
WS2.DLL:	 0
MAPI.DLL:	 0
NETAPI.DLL:	 0
NETSETUP.DLL:	 0
RPCLTC1.DLL:	 0
WSOCK32.DLL:	 0
HYPERTERM.DLL:	 0
MPR.DLL:	 0
MFC30.DLL:	 0
M

### 용어 정리
- DLL: 동적 링크 라이브러리(영어: dynamic-link library, DLL)는 마이크로소프트 윈도우에서 구현된 동적 라이브러리
- section: 우리가 아는 개념!! 섹션을 나누어서 실행 파일에 필요한 정보를 저정
- image: 코드 섹션, 프로그램이 실행 될 때 메모리에 로드되어서 실행됨 쉽게 말해 실행 코드

### 할 수 있는 방안???
1. DLL 파일들은 1인 값만 받아서 set으로 만들기 index [0 ~ 72]
- 이 때 DLL파일이 없으면 조건문으로 뺀다
- 한계점: 72개에 대해서만 minhash값이 나오니 거의다 빅그룹으로 나온다.
	- 차원의 수를 줄이기
	- DLL 파일의 수 늘리기
- 시그니처를 뽑는 방식은 많이 나오는 것이라고 가정
- 이제 한 PE파일을 글러스터링 하기 위해서 피처들을 어떻게 뽑을 것인지?

In [120]:
# 1번째 방안

DLL_ = []

for miner in miner_data:
	DLL_list = []
	for idx in range(73):
		if miner['analy'][miner_key[idx]] != 0:
			DLL_list.append(miner_key[idx])
	if len(DLL_list) != 0:
		DLL_.append(DLL_list)

In [121]:
# check DLL list

DLL_value = set()

for DLL_list in DLL_:
	for DLL in DLL_list:
		DLL_value.add(DLL)
		
print(DLL_value)
len(DLL_value)

{'WININET.DLL', 'COMCTL32.DLL', 'RASAPI32.DLL', 'GDI32.DLL', 'SECUR32.DLL', 'WSOCK32.DLL', 'COMDLG32.DLL', 'SHLWAPI.DLL', 'LZ32.DLL', 'VERSION.DLL', 'OLEDLG.DLL', 'USER32.DLL', 'CRTDLL.DLL', 'KERNEL32.DLL', 'WINMM.DLL', 'SHELL32.DLL', 'NETAPI32.DLL', 'RPCRT4.DLL', 'MPR.DLL'}


19

In [122]:
DLL_[: 5]

[['KERNEL32.DLL', 'USER32.DLL', 'GDI32.DLL', 'SHELL32.DLL'],
 ['KERNEL32.DLL', 'USER32.DLL'],
 ['SHLWAPI.DLL',
  'KERNEL32.DLL',
  'USER32.DLL',
  'COMCTL32.DLL',
  'VERSION.DLL',
  'GDI32.DLL',
  'WINMM.DLL',
  'WSOCK32.DLL',
  'SHELL32.DLL'],
 ['KERNEL32.DLL', 'USER32.DLL'],
 ['SHLWAPI.DLL',
  'KERNEL32.DLL',
  'USER32.DLL',
  'COMCTL32.DLL',
  'VERSION.DLL',
  'GDI32.DLL',
  'WINMM.DLL',
  'WSOCK32.DLL',
  'SHELL32.DLL']]

In [123]:
# hyper prameter

K = 32
M = 2 ** 9
thetaJ = 0.6
# vector_size = 2 ** 9
vector_size = 2 ** 5
# eps = 0.8
eps = 0.4
minpts = 5

In [124]:
# transform origin MV2

def MV2(DLL_data, K, M):
	minhashed_virtual_vectors = []
	for DLL in tqdm(DLL_data):
		encode_pos = minHash(DLL, K) % M
		
		vector = np.zeros(M, dtype=np.int8)
		vector[encode_pos] = 1
		
		minhashed_virtual_vectors.append(vector)

	return minhashed_virtual_vectors


In [125]:
minhashed_virtual_vectors = MV2(DLL_, K, M)

100%|██████████| 24833/24833 [00:08<00:00, 2902.77it/s]


In [126]:
big_group = JIG(minhashed_virtual_vectors, thetaJ)
len(big_group)

checking big group


100%|██████████| 24833/24833 [00:05<00:00, 4306.57it/s]


24820

In [127]:
def SG2(DLL_, vector_size, eps, minpts):
	
	fine_vectors = []
	for DLL_list in tqdm(DLL_):
		vector = np.zeros(vector_size, dtype=np.int8)
		for DLL in DLL_list:
			idx = int(hashlib.md5(DLL.encode()).hexdigest(),16) % vector_size
			vector[idx] += 1;

		fine_vectors.append(vector)

	print('start DBSCAN')
	model = DBSCAN(eps=1-eps, min_samples=minpts, metric='cosine', n_jobs=8)
	model.fit(fine_vectors)
	print('end DBSCAN')

	cluster_labels = model.labels_

	cluster_dict = dict()
	for payload, label in zip(DLL_, cluster_labels):
		if label == -1:
			continue

		if label not in cluster_dict.keys():
			cluster_dict[label] = []
		cluster_dict[label].append(payload)

	DLL_cluster_sig = dict()

	for idx in cluster_dict.keys():
		DLL_count = dict()
		for cluster in cluster_dict[idx]:
			for v in cluster:
				if v not in DLL_count:
					DLL_count[v] = 0
				DLL_count[v] += 1
		DLL_cluster_sig[idx] = DLL_count
		
	return DLL_cluster_sig

In [128]:
big_group_DLL= []

for i in big_group:
    big_group_DLL.append(DLL_[i])

In [129]:
DLL_cluster_sig = SG2(big_group_DLL, vector_size, eps=eps, minpts=minpts)

  0%|          | 0/24820 [00:00<?, ?it/s]

100%|██████████| 24820/24820 [00:00<00:00, 90489.16it/s]


start DBSCAN
end DBSCAN


In [130]:
for idx, DLL_count in DLL_cluster_sig.items():
	print(f'cluster: {idx}')
	DLL_sigs = sorted([(k, v) for k, v in DLL_count.items()], key=lambda x: x[1], reverse=True)
	for sig in DLL_sigs:
		print(sig)
	print('----------------')

cluster: 0
('KERNEL32.DLL', 24750)
('USER32.DLL', 19786)
('SHELL32.DLL', 6334)
('GDI32.DLL', 5343)
('WSOCK32.DLL', 4526)
('VERSION.DLL', 3041)
('COMCTL32.DLL', 2884)
('SHLWAPI.DLL', 2460)
('WININET.DLL', 2421)
('WINMM.DLL', 1319)
('NETAPI32.DLL', 710)
('MPR.DLL', 627)
('LZ32.DLL', 612)
('COMDLG32.DLL', 282)
('RPCRT4.DLL', 112)
('OLEDLG.DLL', 74)
('SECUR32.DLL', 63)
('RASAPI32.DLL', 12)
('CRTDLL.DLL', 1)
----------------


### 실험 결과
- 나올 수 있는 해시 값이 한정적이기 때문에 빅그룹 식별 X
	- 차원을 줄여서 실험 진행
- 클러스터링 전 벡터화에서 벡터의 차원 수에 비해서 인코딩 되는 개수가 너무 적음
	- 벡터의 차원 수를 줄여도 특정 차원의 값이 0인 벡터가 많은 벡터화에서는 크게 의미를 갖지 않음